# Task 4: Demonstrating Quantum Advantage

In [ ]:
"""
This task is intentionally open-ended, so no boilerplate code is provided.

You may use this notebook to develop your solution, or create a separate file if you prefer.
We recommend starting by copying over your previous implementations of the QRNG, TRNG, and PRNG.
Then, explore ways to modularize and combine these components to design a use case that
demonstrates the unique advantages of quantum randomness.

Your write-up can be included directly in this notebook or submitted separately.
You're welcome to prepare it as a Google Doc or LaTeX document and upload a PDF to the GitHub repository—
just be sure to clearly indicate where it can be found if it's not included here.
"""

In [ ]:
# Package Imports - feel free to add what you think might be useful! 
%pip install requests
%pip install numpy
%pip install matplotlib
%pip install pyaudio
%pip install psutil
import requests
import time
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import hashlib
import pyaudio
import struct
import psutil
import platform

In [10]:
def get_qrng_floats(n, min_val=-32768, max_val=32767):
    if n > 128:
        n = 128
    url = "https://qrng.idqloud.com/api/1.0/short"
    headers = {"X-API-KEY": "aTo4BKRvnc49uRWDk034zaua87vGRXKk9TMLdfkI"}
    params = {"min": min_val, "max": max_val, "quantity": n}
    response = requests.get(url, headers=headers, params=params)
    raw = np.array(response.json()['data'], dtype=np.float64)
    scaled = raw - min_val
    scaled = scaled / (abs(max_val) + abs(min_val)) 

    #Random numbers scale to [0,1)
    return scaled

In [ ]:
def get_laplace_transform(P, delta, epsilon):
    #P is the scaled random numbers from get_qrng_floats()
    #Delta is the probability that for any one query, data is leaked (generally 1/N)
    #Epsilon is the threshold for privacy (smaller epsilon = more privacy = less utility)
    b = delta/epsilon
    laplace = []
    for p in P:
        if p == 0:
            laplace.append(float('-inf'))
        elif p < 0.5:
            laplace.append(b * math.log(2 * p))
        else:
            laplace.append(-b * math.log(2 * (1 - p)))
    return laplace

In [6]:
get_qrng_floats(10)

array([0.38355077, 0.42789349, 0.66877241, 0.15347524, 0.04206912,
       0.78310826, 0.12239261, 0.86601053, 0.84707408, 0.74633402])